In [81]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from collections import Counter

import functools

import math

import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

In [2]:
inputFile = "water-treatmennt-original-marked.csv"

dataFrame = pd.read_csv(inputFile, header = 0, sep = ';')
print(dataFrame.shape)
data = dataFrame.values

print("First few data rows:")
print(data[0:10, 0:5])

numSamples = dataFrame.shape[0]
print("Num samples == " + str(numSamples))
numFeatures = dataFrame.shape[1] - 2  #first feature is sampling date and last feature used as class marker
print("Num features == " + str(numFeatures))

regularizationStrength = 10000
learningRate = 0.000001

(527, 40)
First few data rows:
[['D-1/3/90' 44101.0 1.5 7.8 nan]
 ['D-2/3/90' 39024.0 3.0 7.7 nan]
 ['D-4/3/90' 32229.0 5.0 7.6 nan]
 ['D-5/3/90' 35023.0 3.5 7.9 205.0]
 ['D-6/3/90' 36924.0 1.5 8.0 242.0]
 ['D-7/3/90' 38572.0 3.0 7.8 202.0]
 ['D-8/3/90' 41115.0 6.0 7.8 nan]
 ['D-9/3/90' 36107.0 5.0 7.7 215.0]
 ['D-11/3/90' 29156.0 2.5 7.7 206.0]
 ['D-12/3/90' 39246.0 2.0 7.8 172.0]]
Num samples == 527
Num features == 38


In [67]:
##########################################Removal of useless features#####################

Y = dataFrame.iloc[:, -1]  # Class markers
X = dataFrame.iloc[:, 1:-1]  # Features without date and class markers

numClasses = np.amax(Y)
print("Num classes: {0}".format(numClasses))

print(X.iloc[0:10, 0:2])

##########################################Normalization#####################

normalizedX = MinMaxScaler().fit_transform(X)
X = pd.DataFrame(normalizedX)

print(X.iloc[0:10, 0:2])

##########################################Missing data handling#####################

for sampleId in range(X.shape[0]):
    for featureId in range(numFeatures):
        if (math.isnan(X.iloc[sampleId, featureId])):
            X.iloc[sampleId, featureId] = 0.5

Num classes: 13.0
   Q-E      (input flow to plant)   ZN-E     (input Zinc to plant)
0                         44101.0                              1.5
1                         39024.0                              3.0
2                         32229.0                              5.0
3                         35023.0                              3.5
4                         36924.0                              1.5
5                         38572.0                              3.0
6                         41115.0                              6.0
7                         36107.0                              5.0
8                         29156.0                              2.5
9                         39246.0                              2.0
          0         1
0  0.680598  0.041916
1  0.579121  0.086826
2  0.443305  0.146707
3  0.499151  0.101796
4  0.537147  0.041916
5  0.570087  0.086826
6  0.620915  0.176647
7  0.520817  0.146707
8  0.381883  0.071856
9  0.583558  0.056886


In [68]:
def addFeatureForBias(data):
    extendedData = np.zeros((data.shape[0], data.shape[1] + 1))
    extendedData[:, 0:-1] = data
    extendedData[:, -1] = int(1)
    
    return extendedData

# test = pd.DataFrame([[0, 0],
#                      [0, 0]])
# print(addFeatureForBias(test))

X = addFeatureForBias(X)

In [79]:
def computeSoftMarginCost(weights, X, Y):
    # Hinge loss
    N = X.shape[0]
    distances = 1 - Y * (np.dot(X, weights))
    
    print("Num samples: {0}".format(X.shape[0]))
    print("Num classification errors: {0}".format(len(distances[distances < 0])))
    
    distances[distances < 0] = 0  # equivalent to max(0, distance)
    
    dangerousDistances = distances[distances < 1]
    print("Num samples inside borders: {0}".format(len(dangerousDistances[dangerousDistances > 0])))
    
    hinge_loss = regularizationStrength * (np.sum(distances) / N)
    
    # Soft margin cost function
    cost = 1 / 2 * np.dot(weights, weights) + hinge_loss
    return cost

def computeCostFunctionGradient(weights, xBatch, yBatch):
#     print(type(xBatch))
#     print(type(yBatch))
    
    # In case of SGD
    if (type(yBatch) != np.ndarray):
        yBatch = np.array([yBatch])
        xBatch = np.array([xBatch])
        
    distances = 1 - (yBatch * np.dot(xBatch, weights))
    
    weightsDelta = np.zeros(len(weights))
    
    for index, distance in enumerate(distances):
        if (max(0, distance) == 0):
            deltaI = weights
        else:
            deltaI = weights - (regularizationStrength * yBatch[index] * xBatch[index])
            
        weightsDelta += deltaI
        
    weightsDelta = weightsDelta / len(yBatch)
    
    return weightsDelta

def stochasticGradientDescent(features, outputs):
    maxEpochs = 1024
    weights = np.zeros(features.shape[1])

    prevCost = float("inf")
    convergenceThreshold = 0.01 #percents
    
    for epoch in range(maxEpochs): 
        # To prevent repeating update cycles
        X, Y = shuffle(features, outputs)
        
        for ind, x in enumerate(X):
            gradientAscent = computeCostFunctionGradient(weights, x, Y[ind])
            weights = weights - (learningRate * gradientAscent)
            
        if ((epoch % 100 == 0) or (epoch >= maxEpochs - 1)):
            cost = computeSoftMarginCost(weights, features, outputs)
            print("Epoch is:{} and cost is: {}".format(epoch, cost))
            
            if (abs(prevCost - cost) < convergenceThreshold * prevCost):
                return weights
            
            prevCost = cost
            
    return weights

In [82]:
yFirstVsAll = list(map(lambda y: -1.0 if y == 1 else 1.0, Y))

# print(yFirstVsAll)
# print(type(X.values))
# print(type(yFirstVsAll))

X_train, X_test, y_train, y_test = train_test_split(X, yFirstVsAll, test_size=0.3, random_state=42)

print("training started...")
W = stochasticGradientDescent(X_train, y_train)
print("training finished.")
print("weights are: {}".format(W))

y_test_predicted = [0] * X_test.shape[0]

for i in range(X_test.shape[0]):
    yp = np.sign(np.dot(W, X_test[i]))
    y_test_predicted[i] = yp

# print("accuracy on test dataset: {}".format(accuracy_score(y_test, y_test_predicted)))

training started...
Num classification errors: 126
Num samples inside borders: 69
Num samples: 368
Epoch is:0 and cost is: 8993.458591208804
Num classification errors: 161
Num samples inside borders: 128
Num samples: 368
Epoch is:100 and cost is: 5425.699930763443
Num classification errors: 162
Num samples inside borders: 126
Num samples: 368
Epoch is:200 and cost is: 5288.224206491394
Num classification errors: 163
Num samples inside borders: 121
Num samples: 368
Epoch is:300 and cost is: 5249.661323655799
training finished.
weights are: [-0.24127673  1.10670602  0.95850529 -0.49645541  2.13162346  3.48918998
 -2.55778208 -0.51755141  1.02912201  0.82615812 -0.30220422  2.24496249
 -2.01993904 -1.06362738  0.71813307  1.17227739 -1.98080051 -3.8025231
  0.47165857 -1.0426812   1.26436769 -0.42553123  1.93476806 -1.8915434
 -0.78220174  0.77252156  1.38464262 -0.12188401 -0.36968091  2.51988422
  1.09194234 -2.60541872 -0.22571434 -0.11713781  0.0531357  -0.66722472
  0.55046649  0.230